# Genereren van een nieuw metadata bestand

Dit script is voor het genereren van een metadata bestand. Dit bestand is nodig voor het genereren
van de storingsanalyse.

Het genereren van een nieuw metadata bestand is een onderdeel van het toevoegen van een nieuw 
project aan de metadata. Wanneer er al een metadata bestand aanwezig is, wordt deze in het proces
van het genereren van een rapport voor de storings automatoisch geüpdate. 

## Stappen 
Onderstaand worden de stappen uiteengezet die u gevraagd wordt te doorlopen.

### Stap 1 - Specificeren van het project
Metadata bestanden worden geïdentificeerd op basis van de naam van het project. Wanneer een nieuw 
project wordt toegevoegd, is het dus van belang dat u de correcte/officiële projectnaam gebruikt.

#### Voorbeelden
```python
project = "Coentunnel-tracé"
```
```python
project = "Sluis Eefde"
```

In onderstaande code cell moet u de naam van het project specificeren tussen de leestekens, zoals de 
voobeelden weergeven.

In [1]:
project = "Coentunnel-tracé"

### Stap 2 - Bronbestand specificeren
Elk project moet eerst worden toegevoegd aan de automatisering voordat er een storingsanalyse 
gegenereerd kan worden voor het desbetreffende project. Dit 'toevoegen' bestaat uit het genereren 
van een metadata bestand met daarin historische data van het project.

Deze historische data, en de rest van de data, wordt opgehaald uit de Excel bestanden die voorheen 
werden gebruikt bij het handmatig opstellen van de storingsanalyse. U wordt hier gevraagd dit document van de meest recente storingsanalyse te uploaden naar de map ```Automatisering Storingsanalyse/processes/building_metadata_file/source_document```. Zorg dat dit het enige bestand is dat aanwezig is in deze map.

Indien u twijfelt over of u het juiste brondocument heeft, er is een voorbeeld document beschikbaar in de map ```Automatisering Storingsanalyse/documents/example documents``` onder de naam ```brondocument_metadata.xlsx```.

#### Dit is een nieuw project en er is nog nooit een storingsanalyse voor uitgevoerd
Als dit een nieuw project is waar nog nooit een storingsanalyse voor is uitgevoerd, hoeft u geen naam van het brondocument te specificeren. In dat geval kun u onderstaand voorbeeld overnemen.
```python
file_name = None
```

Als u het bestand naar de desbetreffende map heeft geüpload, moet u de naam van dit bestand in onderstaande code cell specificeren tussen de leestekens (net als bij het project).

In [2]:
file_name = "20210505 Storingsdatabase Q1 2021.xlsx"

### Stap 3 - Tijdsregistratie specificeren
Voor sommige projecten is vastgesteld dat zij binnen de storingsanalyse ook een analyse van de tijdsregistratie moeten uitvoeren. Als dit het geval is voor het project dat u wilt toevoegen dan neemt u het eerste voorbeeld over. In het geval dat dit niet van toepassing is, dan neemt u het tweede voorbeeld over.
```python
tijdsregistratie = True
```
```python
tijdsregistratie = False
```

In [3]:
tijdsregistratie = False

### Stap 4 - Starten van het script
Wanneer u alle bovenstaande stappen heeft doorlopen, is het tijd om het script te starten. Dit doet u door te navigeren naar de optie ```Kernel``` in de balk boven aan uw scherm. Vervolgens selecteert u de optie ```Restart & Run All```.

In [4]:
import os

keep_going = True
while keep_going:
    if os.getcwd().endswith("Automatisering Storingsanalyse"):
        keep_going = False
    else:
        os.chdir('..')  # changes the working dir to a level above the current working dir

# Bouwen van de metadata
De metadata is nodig voor het berekeken van de gemiddelde aantal meldingen/storingen per maand en de TOV-tabellen.

De opbouw van de metadata is (vooralsnog) als volgt:
```
{
    project: projectnaam,
    start_datum: dd-mm-yyy,
    contract_info: {
        tijdsregistratie: True,
        minimale_beschikbaarheid: xx,
        minimale_responsetijd: 04:00:00,
    },
    meldingen: {
        f"{maand}_{jaar}": {
            DI_num: aantal meldingen,
            DI_num: aantal meldingen
        }
        f"{maand}_{jaar}": {
            DI_num: aantal meldingen,
            DI_num: aantal meldingen
        }
    },
    storingen: {
        f"{maand}_{jaar}": {
            DI_num: aantal storingen,
            DI_num: aantal storingen
        }
        f"{maand}_{jaar}": {
            DI_num: aantal storingen,
            DI_num: aantal storingen
        }
    }
}
```
In deze opbouw worden enkel de deelinstallaties meegenomen met meldingen > 0. Dit geldt ook voor de storingen.
Om DI_num om te zetten naar een omschrijving kan de functie **get_breakdown_description()** gebruikt worden.

Voor het opslaan van de metadata wordt (voor nu) gekozen voor een .json-type

# Importeren van de benodigdheden

In [5]:
import json
import pandas as pd
import os
import numpy as np
from datetime import datetime

from pandas import DataFrame

# Definieren van de functies

In [6]:
def get_first_key(dictionary: dict) -> dict:
    return list(dictionary.keys())[0]


def del_empty_keys(dictionary: dict) -> dict:
    """
    The tabs that are read have a pre-defined table. This results in some empty dicts with key names that represent
    future months
    :param dictionary:
    :return: Dict without
    """
    return {key: dictionary[key] for key in dictionary.keys() if dictionary[key] != {}}


def clean_dt_string_month(dt_string: str) -> str:
    month_notation = ['Jan', 'Feb', 'Mrt', 'Apr', 'Mei', 'Jun', 'Jul', 'Aug', 'Sept', 'Okt', 'Nov', 'Dec']
    month_notation = {month_notation[idx]: str(idx + 1) for idx in range(len(month_notation))}

    dt_month, dt_year = month_notation[dt_string.split(' - ')[0]], datetime.strptime(dt_string.split(' - ')[1], '%y')

    dt_string = '0' + dt_month + '_' + datetime.strftime(dt_year, '%Y') if len(dt_month) == 1 \
        else dt_month + '_' + datetime.strftime(dt_year, '%Y')

    return dt_string


def clean_dt_string_q(dt_string: str) -> str:
    return dt_string.replace('-', '_').replace(' ', '')


def clean_inputdata(inputdata: DataFrame, index_first_col_maanden: int, category_column_name: str, time_bin: str = 'month') -> dict:
    """
    Gestandaardiseerde aanpak voor het schoonmaken van de input dataframes uit het rekendocument (excel) van
    Remko van Gorkum.
    :param inputdata:
    :param index_first_col_maanden:
    :param category_column_name:
    :return:
    """
    _inputdata = inputdata.iloc[:-3, :]  # onderste 3 rijen zijn overbodig  EDIT (??is dit altijd zo??)
    dictionary = {}
    for col in _inputdata.iloc[:, index_first_col_maanden:]:
        if _inputdata[col][0].lower() == 'totaal':
            break

        datetime_obj = clean_dt_string_month(_inputdata[col][0]) if time_bin == 'month' else clean_dt_string_q(_inputdata[col][0])
        # initialize empty dict for month
        if datetime_obj not in dictionary:
            dictionary[datetime_obj] = {}  # Creates an empty dict w/ month as key in the dict

        for index, row in _inputdata.iterrows():
            if row[col] is np.nan:
                break
            elif index > 0 and int(row[col]) > 0:
                dictionary[datetime_obj][row[category_column_name]] = row[col]

    dictionary = del_empty_keys(dictionary)

    return dictionary

def check_for_file(path_to_folder: str, file: str) -> bool:
    files_in_folder = os.listdir(path_to_folder)
    if file in files_in_folder:
        raise PermissionError(f"A metadata file with the name '{file}' already exists in the specified folder. Executing this code cell would possibly mean that data is lost. Please create a backup of the file and remove it from the folder '{path_to_folder}' before running this process again.")
    else:
        return True

# Uitlezen van het brondocument

In [7]:
path_to_source_folder = 'processes/building_metadata_file/source_document'
excel_file = pd.ExcelFile(os.path.join(path_to_source_folder, file_name), engine='openpyxl')

inputdata_subsystems = pd.read_excel(excel_file, list(filter(lambda x: x.lower() == 'onterechte meldingen totaal', excel_file.sheet_names))[0])
inputdata_poo_codes = pd.read_excel(excel_file, list(filter(lambda x: x.lower() == 'probleem oorzaak oplossing', excel_file.sheet_names))[0])

inputdata_meldingen = pd.read_excel(excel_file, list(filter(lambda x: x.lower() == 'trend maand meldingen', excel_file.sheet_names))[0])
inputdata_storingen = pd.read_excel(excel_file, list(filter(lambda x: x.lower() == 'trend maand storingen', excel_file.sheet_names))[0])

inputdata_poo_probleem = pd.read_excel(excel_file, list(filter(lambda x: x.lower() == 'overzicht probleem', excel_file.sheet_names))[0])
inputdata_poo_oorzaak = pd.read_excel(excel_file, list(filter(lambda x: x.lower() == 'overzicht oorzaak', excel_file.sheet_names))[0])
inputdata_poo_oplossing = pd.read_excel(excel_file, list(filter(lambda x: x.lower() == 'overzicht oplossing', excel_file.sheet_names))[0])

# Transformeren van de brondata

In [8]:
"""
Possible subsystem numbers
"""
possible_subsystems = set()

# Sluis Eefde gebruikt 'SBS subsysteem code'  -  ipv 'SBS sub-systeem code'
column = 'SBS subsysteem code' if project == 'Sluis Eefde' else 'SBS sub-systeem code'
for x in inputdata_subsystems[column][inputdata_subsystems[column].notnull()]:
    possible_subsystems.add(str(x))

In [9]:
"""
meldingen per di_num
di_num = SBS sub-systeem code
"""
meldingen = clean_inputdata(inputdata_meldingen,
                            index_first_col_maanden=4,
                            category_column_name='SBS sub-systeem code',
                            time_bin='month')

In [10]:
"""
storingen per di_num
di_num = SBS sub-systeem code
"""
storingen = clean_inputdata(inputdata_storingen,
                            index_first_col_maanden=4,
                            category_column_name='SBS sub-systeem code',
                            time_bin='month')

In [11]:
"""
POO-codes (Probleem/Oorzaak/Oplossing codes)
"""
poo_probleem = clean_inputdata(inputdata_poo_probleem,
                               index_first_col_maanden=2,
                               category_column_name='Probleem code',
                               time_bin='q')

poo_oorzaak = clean_inputdata(inputdata_poo_oorzaak,
                              index_first_col_maanden=2,
                              category_column_name='Oorzaak code',
                              time_bin='q')

poo_oplossing = clean_inputdata(inputdata_poo_oplossing,
                                index_first_col_maanden=2,
                                category_column_name='Oplossing code',
                                time_bin='q')

poo_codes = {"probleem": poo_probleem,
             "oorzaak": poo_oorzaak,
             "oplossing": poo_oplossing}

poo_code_overzicht = dict()
col_names = ['Probleem', 'Oorzaak', 'Oplossing']
for name in col_names:
    i = inputdata_poo_codes.columns.get_loc(name)
    col_data = inputdata_poo_codes.iloc[:, i].to_dict()
    beschrijving_data = inputdata_poo_codes.iloc[:, i+1].to_dict()

    dict2add = {}
    for idx in range(len(col_data)):
        if col_data[idx] is np.nan:
            break

        if list(col_data.keys())[idx] not in poo_code_overzicht:
            dict2add[col_data[idx]] = beschrijving_data[idx]

    poo_code_overzicht = {**poo_code_overzicht, **dict2add}

# Gereed maken van het JSON object

In [12]:
"""
Set-up van het JSON-Object
"""
contract_info = {"tijdsregistratie": str(tijdsregistratie),
                 "minimale_beschikbaarheid": "xx",
                 "minimale_responsetijd": "04:00:00",
                 "aanwezige_deelinstallaties": tuple(possible_subsystems),
                 "POO_codes": poo_code_overzicht}

start_datum = get_first_key(meldingen)

json_dict = {"project": project,
             "start_datum": start_datum,
             "contract_info": contract_info,
             "poo_codes": poo_codes,
             "meldingen": meldingen,
             "storingen": storingen}

# Exporteren van het JSON object

todo:
    een check met feedback bericht als er al een metadata bestand van het project aanwezig is in de export map. deze moet dan handmatig verwijderd worden en lokaal opgeslagen worden (kan ook geautomatiseerd) om te zorgen dat deze data niet allemaal verloren is wanneer er iets fout gaat o.i.d.

In [13]:
path_to_export_folder = "data/metadata"
export_file_name = "metadata_file_{}.json".format(project.lower().replace(' ', '_'))

check_for_file(path_to_folder=path_to_export_folder, file=export_file_name)

with open(os.path.join(path_to_export_folder, export_file_name), 'w') as output_file:
    json.dump(json_dict, output_file)
    
print('Succes!')

PermissionError: A metadata file with the name 'metadata_file_coentunnel-tracé.json' already exists in the specified folder. Executing this code cell would possibly mean that data is lost. Please create a backup of the file and remove it from the folder 'data/metadata' before running this process again.

# Afronding
Nu het script is uitgevoerd zijn er twee uitkomsten;
    
    1. U ziet de terugkoppeling 'Succes!'.
    2. U ziet een PermissionError met daarbij een terugkoppeling.
    
## Afronding na optie 1
Indien u de terugkoppeling ```Succes!``` ziet, is het script succesvol doorlopen en staat er een niet bestand in de map ```Automatisering Storingsanalyse/data/metadata```.

## Afronding na optie 2
Indien u de PermissionError als terugkoppeling ziet, is het van belang dat u dit bericht volgt en vervolgens het script nogmaals probeert uit te voeren.